In [32]:
import pandas as pd
from pandas.io import gbq
import numpy as np
import os
import glob
import re

In [33]:
import pydata_google_auth

In [34]:
    connect_bq = [
        'https://www.googleapis.com/auth/cloud-platform',
        'https://www.googleapis.com/auth/drive',
    ]

    credentials = pydata_google_auth.get_user_credentials(
        connect_bq,
        auth_local_webserver=True,
    )

#### Query that retrieves add_ons per user for a month and the add ons that a bundle has

In [35]:
sql_query = """
CREATE TEMP FUNCTION STRING_DEDUP(str STRING) AS (
(SELECT STRING_AGG(item ORDER BY item) FROM (
SELECT DISTINCT item FROM UNNEST(SPLIT(str)) item 
)) 
);
----------------------------------------------------------------------------------------------------------------------
with daily_status as (
Select * , DATE_TRUNC(day, month) as change_month
from `fubotv-prod.data_insights.daily_status_static_update` t1
where final_status_restated like '%paid%'
and day >= '2021-01-01'
AND day <= '2022-05-31'
),
add_on as (
select account_code, change_month, STRING_DEDUP(add_ons) as addons_sorted,STRING_DEDUP(bundle_add_on_list) as bundle_addons_sorted
FROM daily_status
)
select DISTINCT change_month as month, account_code, bundle_addons_sorted, addons_sorted
from add_on
"""

In [36]:
df = pd.read_gbq(
    sql_query,
    project_id='fubotv-prod',
    credentials=credentials,
)

#### Checking / Housekeeping

In [37]:
df.head()

,month,account_code,bundle_addons_sorted,addons_sorted
0,2021-03-01,5fc0f7662c1aa6000199322c,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,fubo-extra-lite,thi..."
1,2021-12-01,60073be7b4d390000150ebc9,"advanced-dvr-250,third-screen","advanced-dvr-250,premiere,third-screen"
2,2021-09-01,5709557d1dd0e50100c5702d,None,rsn-fee
3,2021-06-01,5c08249bb2463f0007d852cf,advanced-dvr-250,"advanced-dvr-250,premiere"
4,2021-12-01,5f77eda7d8bf910001d93588,"advanced-dvr-1000,sports-lite,third-screen,unl...","advanced-dvr-1000,sports-lite,third-screen,unl..."


##### if the bundles addon column is empty, discard that row

In [38]:
df = df.dropna()

##### Comparing bundle add_ons to the add_ons field and identifying the difference between these two fields and joining each add_on by a comma to a new field called new_addition

In [39]:
df['nonbundle_addons'] = [','.join(set(n.split(',')) - set(o.split(','))) 
                                                          for o, n in zip(df.bundle_addons_sorted, df.addons_sorted)]

In [40]:
df

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2021-03-01,5fc0f7662c1aa6000199322c,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,fubo-extra-lite,thi...","adventure,fubo-extra-lite"
1,2021-12-01,60073be7b4d390000150ebc9,"advanced-dvr-250,third-screen","advanced-dvr-250,premiere,third-screen",premiere
3,2021-06-01,5c08249bb2463f0007d852cf,advanced-dvr-250,"advanced-dvr-250,premiere",premiere
4,2021-12-01,5f77eda7d8bf910001d93588,"advanced-dvr-1000,sports-lite,third-screen,unl...","advanced-dvr-1000,sports-lite,third-screen,unl...",
5,2021-09-01,5bccc16821fac9000351c1a6,"advanced-dvr,third-screen","advanced-dvr,deportes,rsn-fee,third-screen","deportes,rsn-fee"
...,...,...,...,...,...
17337684,2022-05-01,5e5bfe54d1622b0001938760,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337685,2022-01-01,5fed2f69142fd70001caa60a,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337686,2021-10-01,614f5e5602d4e70001a5f167,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337687,2021-12-01,6193ecd7e39e1d000182f649,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee


#### Splitting new_addition column into additional columns and exploding them into individual rows

In [41]:
df['nonbundle_addons'] = df['nonbundle_addons'].str.split(',')
df = df.explode('nonbundle_addons').reset_index(drop=True)
cols = list(df.columns)
df = df[cols]

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17748722 entries, 0 to 17748721
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   month                 datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
 4   nonbundle_addons      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 677.1+ MB


In [43]:
df = df.drop(['addons_sorted', 'bundle_addons_sorted'], axis=1)

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2021-03-01,5fc0f7662c1aa6000199322c,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,fubo-extra-lite,thi...",adventure
1,2021-03-01,5fc0f7662c1aa6000199322c,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,fubo-extra-lite,thi...",fubo-extra-lite
2,2021-12-01,60073be7b4d390000150ebc9,"advanced-dvr-250,third-screen","advanced-dvr-250,premiere,third-screen",premiere
3,2021-06-01,5c08249bb2463f0007d852cf,advanced-dvr-250,"advanced-dvr-250,premiere",premiere
4,2021-12-01,5f77eda7d8bf910001d93588,"advanced-dvr-1000,sports-lite,third-screen,unl...","advanced-dvr-1000,sports-lite,third-screen,unl...",


In [44]:
df = df.drop_duplicates()

In [45]:
print(df)

              month              account_code nonbundle_addons
0        2021-03-01  5fc0f7662c1aa6000199322c        adventure
1        2021-03-01  5fc0f7662c1aa6000199322c  fubo-extra-lite
2        2021-12-01  60073be7b4d390000150ebc9         premiere
3        2021-06-01  5c08249bb2463f0007d852cf         premiere
4        2021-12-01  5f77eda7d8bf910001d93588                 
...             ...                       ...              ...
17748717 2022-05-01  5e5bfe54d1622b0001938760          rsn-fee
17748718 2022-01-01  5fed2f69142fd70001caa60a          rsn-fee
17748719 2021-10-01  614f5e5602d4e70001a5f167          rsn-fee
17748720 2021-12-01  6193ecd7e39e1d000182f649          rsn-fee
17748721 2021-02-01  5f79fea032e2d800012406a9          rsn-fee

[17748722 rows x 3 columns]


In [46]:
df

,month,account_code,nonbundle_addons
0,2021-03-01,5fc0f7662c1aa6000199322c,adventure
1,2021-03-01,5fc0f7662c1aa6000199322c,fubo-extra-lite
2,2021-12-01,60073be7b4d390000150ebc9,premiere
3,2021-06-01,5c08249bb2463f0007d852cf,premiere
4,2021-12-01,5f77eda7d8bf910001d93588,
...,...,...,...
17748717,2022-05-01,5e5bfe54d1622b0001938760,rsn-fee
17748718,2022-01-01,5fed2f69142fd70001caa60a,rsn-fee
17748719,2021-10-01,614f5e5602d4e70001a5f167,rsn-fee
17748720,2021-12-01,6193ecd7e39e1d000182f649,rsn-fee


###### Remove RSN fee and blanks from the field

In [47]:
df = df[df.nonbundle_addons != 'rsn-fee']
df = df[df.nonbundle_addons != '']
df = df[df.nonbundle_addons != ""]

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4392679 entries, 0 to 17748705
Data columns (total 3 columns):
 #   Column            Dtype         
---  ------            -----         
 0   month             datetime64[ns]
 1   account_code      object        
 2   nonbundle_addons  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 134.1+ MB


In [49]:
df.head()

,month,account_code,nonbundle_addons
0,2021-03-01,5fc0f7662c1aa6000199322c,adventure
1,2021-03-01,5fc0f7662c1aa6000199322c,fubo-extra-lite
2,2021-12-01,60073be7b4d390000150ebc9,premiere
3,2021-06-01,5c08249bb2463f0007d852cf,premiere
5,2021-09-01,5bccc16821fac9000351c1a6,deportes


###### Converting timestamp to Date

In [50]:
df['month'] = pd.to_datetime(df['month'], format="%Y/%m/%d").dt.date

#### Account code level Table

In [52]:
df.to_gbq ( destination_table='business_analytics.temp_nonbundle_addon_account_code', project_id= 'fubotv-dev', if_exists= 'replace', table_schema = [{'name':'month','type': 'DATE'}])

#### Aggregating by month and new_addition to count distinct account_code

In [53]:
df = df.groupby(['month', 'nonbundle_addons'],  as_index=False ).agg({'account_code': 'nunique'})

In [55]:
df

,month,nonbundle_addons,account_code
0,2021-01-01,advanced-dvr,11215
1,2021-01-01,advanced-dvr-1000,3508
2,2021-01-01,advanced-dvr-250,56869
3,2021-01-01,adventure,6088
4,2021-01-01,amc-premiere,2725
...,...,...,...
486,2022-05-01,starz,2232
487,2022-05-01,starz-epix-showtime,7516
488,2022-05-01,third-screen,3183
489,2022-05-01,tv5-monde,807


In [54]:
df.to_gbq ( destination_table='business_analytics.temp_nonbundle_addon_counts', project_id= 'fubotv-dev', if_exists= 'replace', table_schema = [{'name':'month','type': 'DATE'}])